In [1]:
import argparse
from logging import getLogger
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import pickle

from recbole.quick_start import load_data_and_model
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color
from recbole.data import create_dataset
from recbole.utils.case_study import full_sort_topk, full_sort_scores

import logging
from logging import getLogger

import torch
import pickle

from recbole.config import Config
from recbole.data import create_dataset, data_preparation, save_split_dataloaders, load_split_dataloaders
from recbole.utils import init_logger, get_model, get_trainer, init_seed, set_color

import copy
from recbole.data.interaction import Interaction, cat_interactions

pd.set_option('display.max_rows', None)


In [2]:
model_file = 'saved/SASRec-task-265-Dec-11-2021_19-52-36.pth'

checkpoint = torch.load(model_file)
config = checkpoint['config']
# config.final_config_dict['load_col']['inter'].append('cityid')
# config.final_config_dict['unused_col']['inter'] = ['exposure_time', 'exposure_time_list']
# config.final_config_dict['save_dataset'] = True
# config.final_config_dict['save_dataloaders'] = True

init_seed(config['seed'], config['reproducibility'])
init_logger(config)
logger = getLogger()
logger.info(config)

dataset = MetaSeqDataset(config)
logger.info(dataset)

train_data, test_data = dataset.build()
train_data = MetaLearningDataLoader(config, dataset, train_data)
test_data = MetaLearningDataLoader(config, dataset, test_data)

init_seed(config['seed'], config['reproducibility'])
model = get_model(config['model'])(config, train_data.dataset).to(config['device'])
model.load_state_dict(checkpoint['state_dict'])
model.load_other_parameter(checkpoint.get('other_parameter'))

trainer = MetaLearningTrainer(config, model)

RuntimeError: CUDA error: out of memory